In [1]:
import numpy as np
import pandas as pd
import numpy.linalg as la
from random import random
from random import seed

# Load the data

In [2]:
datadir = '../data/processed/'
train_file = datadir + 'train.csv'
train = pd.read_csv(train_file)

# Create network

In [3]:
def initialize_network(n_inputs, n_hidden, n_output=1):
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs+1)]} \
                    for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden+1)]}]
    network.append(output_layer)
    return network

In [4]:
network = initialize_network(2, 2)

In [5]:
for layer in network:
    print(layer)

[{'weights': [0.6872817227251943, 0.2999694717949647, 0.10105461524770576]}, {'weights': [0.7750804838718501, 0.9075508353070193, 0.9805961012131525]}]
[{'weights': [0.39303531319072715, 0.3118825792043687, 0.6856634540480872]}]


# Forward propagation

## Neuron activation

In [6]:
def activate(weights, inputs):
    # Add the bias term
    activation = weights[-1]
    # Add the product of all the other weights and inputs
    for i in range(len(inputs)):
        activation += weights[i] * inputs[i]
    return activation

## Transfer

In [7]:
def transfer(activation):
    return 1.0 / (1.0 + np.exp(-activation))

## Forward propagation

In [8]:
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [9]:
network

[[{'weights': [0.6872817227251943, 0.2999694717949647, 0.10105461524770576]},
  {'weights': [0.7750804838718501, 0.9075508353070193, 0.9805961012131525]}],
 [{'weights': [0.39303531319072715, 0.3118825792043687, 0.6856634540480872]}]]

In [10]:
row = [1, 0]
output = forward_propagate(network, row)
print(output)

[0.7723796521479426]


# Back propagate error

## Transfer derivative

In [11]:
def transfer_derivative(output):
    return output * (1.0 - output)

## Error backpropagation

In [12]:
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [13]:
network

[[{'weights': [0.6872817227251943, 0.2999694717949647, 0.10105461524770576],
   'output': 0.6874739985056177},
  {'weights': [0.7750804838718501, 0.9075508353070193, 0.9805961012131525],
   'output': 0.8526673557281667}],
 [{'weights': [0.39303531319072715, 0.3118825792043687, 0.6856634540480872],
   'output': 0.7723796521479426}]]

In [14]:
expected = [0]
backward_propagate_error(network, expected)

In [15]:
network

[[{'weights': [0.6872817227251943, 0.2999694717949647, 0.10105461524770576],
   'output': 0.6874739985056177,
   'delta': -0.011466918761383005},
  {'weights': [0.7750804838718501, 0.9075508353070193, 0.9805961012131525],
   'output': 0.8526673557281667,
   'delta': -0.005320377740051228}],
 [{'weights': [0.39303531319072715, 0.3118825792043687, 0.6856634540480872],
   'output': 0.7723796521479426,
   'delta': -0.13579154536183213}]]

# Train network

## Update weights

In [16]:
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

## Train network

In [17]:
def train_network(network, train, l_rate, n_epoch, n_outputs=1):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        print(f'>epoch={epoch}, lrate={l_rate:.3f}, error={sum_error:.3f}')

# Test the code

In [18]:
seed(3)

In [19]:
n_inputs = train.values.shape[1] - 1 
n_inputs

57

In [20]:
n_outputs = 1

In [21]:
network = initialize_network(n_inputs, 5, 1)

In [22]:
train_network(network, train.values, 0.5, 20, n_outputs)

TypeError: list indices must be integers or slices, not numpy.float64

# Maybe more object oriented?

In [31]:
class Neuron():
    def __init__(self, n_inputs):
        self.weights = np.random.random(n_inputs+1)
        self.out = None
        self.error = None
        self.delta = None
    
    def predict(self, x):
        tot = self.weights[-1]
        # multiply weights times x
        for ix in len(x):
            tot += x[ix] * self.weights[ix]
        # call activate on product
        return self.activate(tot)
    
    def activate(self, x):
        self.out = 1 / (1 - np.exp(-x))
        return self.out

In [32]:
class Layer():
    def __init__(self, n_inputs, n_neurons):
        self.neurons = [Neuron(n_inputs) for _ in range(n_neurons)]
    
    def predict(self, x):
        res = [n.transfer(x) for n in self.neurons]
        return res

In [33]:
class NN():
    def __init__(self, n_inputs, n_hidden, hidden_width, n_outputs):
        self.input = Layer(n_inputs, hidden_width)
        self.hiddens = [Layer(hidden_width, hidden_width) for i in range(n_hidden)]
        self.output = Layer(hidden_width, 1)
    
    def predict(self, x):
        tmp = self.input.predict(x)
        for h in self.hiddens:
            tmp = h.predict(tmp)
        tmp = self.output.predict(tmp)
        return tmp

In [ ]:
net = NN()